# 기본 스태킹 모델

In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier # 개별 모델(1)
from sklearn.ensemble import RandomForestClassifier # 개별 모델(2)
from sklearn.ensemble import AdaBoostClassifier # 개별 모델(3)
from sklearn.tree import DecisionTreeClassifier # 개별 모델(4)
from sklearn.linear_model import LogisticRegression # 최종 모델

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=0.2, random_state=0)
# stratify=y_label

In [3]:
# 개별 ML 모델 생성
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

# 최종 ML 모델 생성
lr_clf = LogisticRegression(C=10)

In [4]:
# 개별 ML 모델 학습
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

In [5]:
# 개별 ML 모듈의 정확도 측정
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print("KNN 정확도 : {:.4f}".format(accuracy_score(y_test, knn_pred)))
print("랜덤 포레스트 정확도 : {:.4f}".format(accuracy_score(y_test, rf_pred)))
print("결정 트리 정확도 : {:.4f}".format(accuracy_score(y_test, dt_pred)))
print("에이다부스트 정확도 : {:.4f}".format(accuracy_score(y_test, ada_pred)))

KNN 정확도 : 0.9211
랜덤 포레스트 정확도 : 0.9649
결정 트리 정확도 : 0.9123
에이다부스트 정확도 : 0.9561


In [6]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
pred.shape

(4, 114)

In [7]:
# 행, 열 교환
pred = np.transpose(pred)
pred.shape

(114, 4)

In [8]:
# 최종 ML 모듈 학습, 정확도 측정
lr_clf.fit(pred, y_test)
final_lr_pred = lr_clf.predict(pred)

print("최종 로지스틱 회귀 정확도 : {:.4f}".format(accuracy_score(y_test, final_lr_pred)))

최종 로지스틱 회귀 정확도 : 0.9737


# CV 세트 기반의 스태킹

In [9]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error # 오차를 절대값으로 바꿔서 평균 구함

In [10]:
# 최종 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_fold):
    # KFold 생성
    kf = KFold(n_splits=n_fold, shuffle=False, random_state=0)
    # 최종 모델이 사용할 학습용 데이터가 들어갈 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_fold))
    print(model.__class__.__name__, 'model 시작')
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        # 원본 학습 데이터 - 학습/검증 데이터 세트 추출
        print("\t 폴드 세트 : ", folder_counter, "시작")
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        # 만들어진 학습 데이터 - 학습 수행
        model.fit(X_tr,y_tr)
        # 만들어진 검증 데이터 - 예측, 데이터 저장
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        # 원본 테스트 데이터 - 예측, 데이터 저장
        test_pred[:, folder_counter] = model.predict(X_test_n)
        
    # 테스트 데이터 생성
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)
    # 최종 모델이 사용할 학습 데이터, 테스트 데이터 출력
    return train_fold_pred, test_pred_mean

In [11]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


KNeighborsClassifier model 시작
	 폴드 세트 :  0 시작
	 폴드 세트 :  1 시작
	 폴드 세트 :  2 시작
	 폴드 세트 :  3 시작
	 폴드 세트 :  4 시작
	 폴드 세트 :  5 시작
	 폴드 세트 :  6 시작
RandomForestClassifier model 시작
	 폴드 세트 :  0 시작
	 폴드 세트 :  1 시작
	 폴드 세트 :  2 시작
	 폴드 세트 :  3 시작
	 폴드 세트 :  4 시작
	 폴드 세트 :  5 시작
	 폴드 세트 :  6 시작
DecisionTreeClassifier model 시작
	 폴드 세트 :  0 시작
	 폴드 세트 :  1 시작
	 폴드 세트 :  2 시작
	 폴드 세트 :  3 시작
	 폴드 세트 :  4 시작
	 폴드 세트 :  5 시작
	 폴드 세트 :  6 시작
AdaBoostClassifier model 시작
	 폴드 세트 :  0 시작


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


	 폴드 세트 :  1 시작
	 폴드 세트 :  2 시작
	 폴드 세트 :  3 시작
	 폴드 세트 :  4 시작
	 폴드 세트 :  5 시작
	 폴드 세트 :  6 시작


In [12]:
print(X_train.shape)
print(X_test.shape)
print(ada_train.shape)
print(ada_test.shape)

(455, 30)
(114, 30)
(455, 1)
(114, 1)


In [13]:
stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)
print(X_train.shape)
print(X_test.shape)
print(stack_final_X_train.shape)
print(stack_final_X_test.shape)

(455, 30)
(114, 30)
(455, 4)
(114, 4)


In [14]:
# 최종 ML 모듈 학습, 정확도 측정
lr_clf.fit(stack_final_X_train, y_train)
stack_final = lr_clf.predict(stack_final_X_test)

print("최종 로지스틱 회귀 정확도 : {:.4f}".format(accuracy_score(y_test, stack_final)))

최종 로지스틱 회귀 정확도 : 0.9737
